In [1]:
import numpy as np
import pywt
import sys
sys.path.append('../input')

from isrucutitls.Utils import *
from isrucutitls.DataGenerator import *

Path = "../input/isruc-s3-preprocess/ISRUC_S3_all.npz"
# Path = "../input/isruc-s3-wavelet-process/ISRUC_S3_wavelet_process.npz"
ReadList = np.load(Path, allow_pickle=True)
Fold_Num   = ReadList['Fold_len']    # Num of samples of each fold
Fold_Data  = ReadList['Fold_data']   # Data of each fold
Fold_Label = ReadList['Fold_label']  # Labels of each fold

print("Read data successfully")
print('Number of samples: ',np.sum(Fold_Num))

Read data successfully
Number of samples:  8589


In [2]:
DataGenerator = kFoldGenerator(Fold_Data, Fold_Label)
Dom_Generator = DominGenerator(Fold_Num)
del Fold_Data

In [3]:
Fold_Num

array([924, 911, 794, 764, 914, 823, 784, 970, 939, 766], dtype=int32)

In [4]:
train_domin, val_domin = Dom_Generator.getFold(1)

In [5]:
sum(train_domin)

array([924, 794, 764, 914, 823, 784, 970, 939, 766])

In [6]:
sum(val_domin)

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [7]:
# !pip install wfdb
# import wfdb
import pandas as pd
import numpy as np
import os
from scipy.interpolate import splev, splrep
import pickle
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
# !pip install biosppy
# import biosppy.signals.tools as st

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D, Activation,\
BatchNormalization, Add, Reshape, TimeDistributed, Input, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.utils import Sequence

# !pip install torch
import torch
# import torch.nn as nn
# !pip install torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import torchaudio
# !pip install librosa
# import librosa

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, scale
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from tqdm.auto import tqdm
from tensorflow.python.client import device_lib
import pywt

import random as python_random
np.random.seed(4242)
python_random.seed(4242)
tf.random.set_seed(4242)
print("keras version:", keras.__version__)
 
print(device_lib.list_local_devices())


strategy = tf.distribute.MirroredStrategy()
# tf.config.experimental.list_physical_devices('CPU')
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
# AUTO = tf.data.experimental.AUTOTUNE

# # Create strategy from tpu
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# strategy = tf.distribute.experimental.TPUStrategy(tpu)

keras version: 2.6.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16031863231458860711
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15696920576
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2672983201434185995
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


2022-05-30 13:11:32.540803: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-30 13:11:32.611321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-30 13:11:32.612619: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-30 13:11:32.613587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Number of devices: 1


In [8]:
def CNN(inputs, fs=32, kernel_size=25, pool_size=16, weight=0.001):
    x = Conv1D(fs, kernel_size,1, padding='same', kernel_regularizer=l2(weight))(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size, 2, padding='same')(x)
    return x

def ResNet(inputs, fs=32, ks_1=25, ps_1=16, ks_2=25, ps_2=16, weight=0.001):
    x = CNN(inputs, fs, ks_1, ps_1,weight)
    x = CNN(x, fs, ks_2, ps_2,weight)
    shortcut_x = Conv1D(fs,1,2,padding='same')(inputs)
    shortcut_x = Conv1D(fs,1,2,padding='same')(shortcut_x)
    return Add()([x, shortcut_x])

## CL

In [9]:
# 注意睡眠期 N1，N2，N3 可能很相似，因此需要调低 温度系数
temperature = 0.07
from tensorflow.keras import backend as K
import tensorflow as tf
def multi_label_supconloss(features, labels):
    """
    :param features: (batch_size, feature_dim)
    :param labels: (batch_size, label_dim), labels should be one-hot vector
    :return: loss
    """
    batch_size = tf.shape(features)[0]
    features = K.l2_normalize(K.cast(features, dtype=tf.float64), axis=1)

    # compute the cosine similarity, we assume the features were normalized after embedding
    logit = -K.dot(features, K.transpose(features)) / temperature # (batch_size, batch_size)
    # for numerical stability
    logits_max = K.max(logit, axis=1, keepdims=True)
    logit = logit - logits_max
    # compute the euclidean distance
    # logit = K.sum(tf.square(features), axis=1, keepdims=True) - 2 * K.dot(features, K.transpose(features)) + \
    #         K.transpose(K.sum(K.square(features), axis=1, keepdims=True))  # (batch_size, batch_size)
    # logit = K.sqrt(K.cast(logit, dtype=tf.float64)) / temperature

    # mask self-contrast similarity
    # logit_mask = K.ones((batch_size, batch_size), dtype=tf.bool) - K.eye(batch_size, dtype=tf.bool)
    logit_mask = K.cast(~tf.eye(batch_size, dtype=tf.bool), dtype=tf.float64)
    exp_logit = K.exp(logit) * logit_mask

    log_prob = logit - K.log(K.sum(exp_logit, axis=1, keepdims=True))
    
    # compute the weight for each pair(i,j)'s similarity
    weight = K.cast(K.dot(labels, K.transpose(labels)), dtype=tf.float64)
    # mask
    weight_mask = K.cast(~tf.eye(batch_size, dtype=bool), dtype=tf.float64)
    weight = weight * weight_mask
    # weight = -weight / K.sum(weight, axis=1, keepdims=True)
    total_weight = K.sum(weight, axis=1, keepdims=True)
    weight = tf.where(total_weight != 0., weight / total_weight, weight)
    loss = K.mean(K.cast(weight, dtype=tf.float64) * log_prob)
    return -loss

## Create Model

In [10]:
class GradientReversal(keras.layers.Layer):
    '''Flip the sign of gradient during training.'''

    @tf.custom_gradient
    def grad_reverse(self, x):
        y = tf.identity(x)
        def custom_grad(dy):
            return -self.hp_lambda * dy
        return y, custom_grad

    # --------------------------------------
    def __init__(self, hp_lambda=0.0001, **kwargs):
        super(GradientReversal, self).__init__(**kwargs)
        self.hp_lambda = K.variable(hp_lambda, dtype='float32', name='hp_lambda')

    # --------------------------------------
    def call(self, x, mask=None):
        return self.grad_reverse(x)

    # --------------------------------------
    def set_hp_lambda(self,hp_lambda):
        #self.hp_lambda = hp_lambda
        K.set_value(self.hp_lambda, hp_lambda)

    # --------------------------------------
    def increment_hp_lambda_by(self,increment):
        new_value = float(K.get_value(self.hp_lambda)) +  increment
        K.set_value(self.hp_lambda, new_value)

    def get_hp_lambda(self):
        return float(K.get_value(self.hp_lambda))
    def get_config(self):
        config = {}
        base_config = super(GradientReversal, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def create_model(input_shape, channels=10, time_second=30, freq=100,num_domain=9):
    inputs_channel = Input(shape=(time_second*freq, 1))
    x = ResNet(inputs_channel, 16)
    x = Dropout(0.2)(x)
    x = ResNet(x, 32)
    x = Dropout(0.2)(x)
    x = ResNet(x, 64)
    x = Dropout(0.2)(x)
    x = ResNet(x, 128)
#     x = tf.keras.layers.MultiHeadAttention(num_heads=2,key_dim=2)(x, x)
#     x = ResNet(x, 10, 6, 6, 4)
#     x = ResNet(x, 5, 3, 4, 2)
#     x = Flatten()(x)
#     outputs = Dense(128)(x)
    outputs = GlobalAveragePooling1D()(x)
    
    fea_part = Model(inputs=inputs_channel, outputs=outputs)
    fea_part.summary()
    
    # extract the features from each channel
    inputs = Input(shape=input_shape)
    input_re = Reshape((channels, time_second * freq, 1))(inputs)
#     fea_all = tf.stack([fea_part(input_re[:,i,:,:]) for i in range(channels)], axis=1)
    fea_all = TimeDistributed(fea_part)(input_re)
#     fea_all = tf.keras.layers.Attention(use_scale=True)([fea_all, fea_all])
    
    fla_fea = Flatten()(fea_all)
    fla_fea = Dropout(0.5)(fla_fea)
#     merged = GlobalAveragePooling1D()(fea_all)
    merged = Dense(128, name='Feature')(fla_fea)
    label_out = Dense(5, activation='softmax', name='Label')(merged)
    # GRL 
    GRL_in = GradientReversal()(merged)
#     dense_out = Dense(128)(GRL_in)
    domain_out = Dense(num_domain, activation='softmax', name='Domain')(GRL_in)
    
    fea_model = Model(inputs, fea_all)
    cl_model = Model(inputs, merged)
    ce_model = Model(inputs, [label_out, domain_out, merged])
    pre_model = Model(inputs, label_out)
    
    return fea_model, cl_model, ce_model, pre_model


def model_test():
    train_data, val_data = [np.random.rand(1,3000,10), np.random.rand(1,3000,10)]
    fea_model, cl_model, ce_model, pre_model = create_model(train_data.shape[1:])
    cl_model.summary()
    ce_model.summary()
    fea_model.summary()
    pre_model.summary()
model_test()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 3000, 16)     416         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 3000, 16)     64          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 3000, 16)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
import gc
# k-fold cross validation
all_scores = []

cfg = {
    'bs':128,
    'epochs': 50
}
first_decay_steps = 10
lr_decayed_fn = (
  tf.keras.optimizers.schedules.CosineDecayRestarts(
      0.001,
      first_decay_steps))

tf.config.experimental_run_functions_eagerly(True)

for i in range(0, 10):
    print('Fold #', i)
    train_data,train_targets,val_data,val_targets = DataGenerator.getFold(i)
    train_data, val_data = train_data.reshape(-1, 3000, 6+4), val_data.reshape(-1, 3000, 6+4)
    train_domin, val_domin = Dom_Generator.getFold(i)
#     with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, amsgrad=True)
    fea_model, cl_model, ce_model, pre_model = create_model(train_data.shape[1:])
    cl_model.compile(optimizer=opt, 
                  loss=multi_label_supconloss)
    verbose=0
    if i == 0:
        verbose=1
    history = cl_model.fit(train_data, train_targets, 
                    batch_size=cfg['bs'], epochs=15, 
                    validation_data=(val_data, val_targets),
                    verbose=verbose)
#     with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, amsgrad=True)
    ce_model.compile(optimizer=opt, 
                     loss={'Label': "categorical_crossentropy", 'Domain': "categorical_crossentropy"}, 
                     metrics={'Label': "acc", 'Domain': "acc"}, 
                     loss_weights=[1,0.1]
                  )
    pre_model.compile(optimizer=opt, 
                  loss="categorical_crossentropy", 
                  metrics=['acc'], 
                  )
    
    history = ce_model.fit(train_data, [train_targets, train_domin, train_targets], 
                           batch_size=cfg['bs'], epochs=30, 
                           validation_data=(val_data, [val_targets, val_domin, val_targets]),
                           callbacks=[tf.keras.callbacks.ModelCheckpoint(str(i)+'ResNet_Best'+'.h5', 
                                                   monitor='val_Label_acc', 
                                                   verbose=0, 
                                                   save_best_only=True, 
                                                   save_weights_only=True, 
                                                   mode='auto', 
                                                   period=1 )],
                           verbose=verbose)
    # get and save the learned feature
    ce_model.load_weights(str(i)+'ResNet_Best'+'.h5')
    train_feature = fea_model.predict(train_data)
    val_feature = fea_model.predict(val_data)
    print(val_feature.shape)
    np.savez('Feature_'+str(i)+'.npz', 
        train_feature = train_feature,
        val_feature = val_feature,
        train_targets = train_targets,
        val_targets = val_targets
    )
    keras.backend.clear_session()
    del train_data,train_targets,val_data,val_targets, fea_model, cl_model, train_domin, val_domin
    gc.collect()

Fold # 0
Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 3000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_16 (Conv1D)              (None, 3000, 16)     416         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 3000, 16)     64          conv1d_16[0][0]                  
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 3000, 16)     0           batch_normalization_8[0][0]      
___________________________________________________________________________________

2022-05-30 13:11:44.098065: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 919800000 exceeds 10% of free system memory.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "
2022-05-30 13:11:45.284315: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 919800000 exceeds 10% of free system memory.
2022-05-30 13:11:46.065686: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-05-30 13:11:47.038974: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


60/60 [==============================] - 38s 493ms/step - loss: 0.1544 - val_loss: 0.0941
Epoch 2/15
60/60 [==============================] - 27s 450ms/step - loss: 0.0749 - val_loss: 0.0876
Epoch 3/15
60/60 [==============================] - 28s 468ms/step - loss: 0.0703 - val_loss: 0.0843
Epoch 4/15
60/60 [==============================] - 28s 468ms/step - loss: 0.0650 - val_loss: 0.0748
Epoch 5/15
60/60 [==============================] - 28s 469ms/step - loss: 0.1157 - val_loss: 0.1613
Epoch 6/15
60/60 [==============================] - 28s 467ms/step - loss: 0.1284 - val_loss: 0.1232
Epoch 7/15
60/60 [==============================] - 27s 449ms/step - loss: 0.1025 - val_loss: 0.1064
Epoch 8/15
60/60 [==============================] - 28s 472ms/step - loss: 0.0893 - val_loss: 0.0963
Epoch 9/15
60/60 [==============================] - 28s 468ms/step - loss: 0.0948 - val_loss: 0.0998
Epoch 10/15
60/60 [==============================] - 28s 468ms/step - loss: 0.0820 - val_loss: 0.0889


...

(970, 10, 128)
Fold # 8
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 3000, 16)     416         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 3000, 16)     64          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 3000, 16)     0           batch_normalization[0][0]        
______________________________________________________________________

2022-05-30 16:06:48.417363: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.94GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-05-30 16:26:59.390902: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.48GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


(939, 10, 128)
Fold # 9
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3000, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 3000, 16)     416         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 3000, 16)     64          conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 3000, 16)     0           batch_normalization[0][0]        
______________________________________________________________________

2022-05-30 16:27:32.193423: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.41GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-05-30 16:27:32.211069: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.59GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


(766, 10, 128)


In [12]:
res = []
for i in range(10):
    train_data,train_targets,val_data,val_targets = DataGenerator.getFold(i)
    val_data = val_data.reshape(-1, 3000, 6+4)
    ce_model.load_weights(str(i)+'ResNet_Best'+'.h5')
    val_mse, val_acc = pre_model.evaluate(val_data, val_targets, verbose=0)
    res.append(val_acc)
    print('Evaluate', val_acc)
    del train_data,train_targets,val_data,val_targets

Evaluate 0.8571428656578064
Evaluate 0.8221734166145325
Evaluate 0.8438287377357483
Evaluate 0.7526177763938904
Evaluate 0.863238513469696
Evaluate 0.8153098225593567
Evaluate 0.8571428656578064
Evaluate 0.907216489315033
Evaluate 0.813631534576416
Evaluate 0.8015666007995605


In [13]:
np.array(res).mean()

0.8333868622779846

In [14]:
res

[0.8571428656578064,
 0.8221734166145325,
 0.8438287377357483,
 0.7526177763938904,
 0.863238513469696,
 0.8153098225593567,
 0.8571428656578064,
 0.907216489315033,
 0.813631534576416,
 0.8015666007995605]